In [1]:
import torch
import torch.nn as nn
torch.__version__

'1.5.1'

In [2]:
import torch.nn.functional as F

In [3]:
class Net(nn.Module):
    def __init__(self):
        # nn.Module子类的函数必须在构造函数中执行父类的构造函数
        super(Net, self).__init__()
        
        # 卷积层 '1'表示输入图片为单通道， '6'表示输出通道数，'3'表示卷积核为3*3
        self.conv1 = nn.Conv2d(1, 6, 3) 
        #线性层，输入1350个特征，输出10个特征
        self.fc1   = nn.Linear(1350, 10)  #这里的1350是如何计算的呢？这就要看后面的forward函数
    #正向传播 
    def forward(self, x): 
        print(x.size()) # 结果：[1, 1, 32, 32]
        # 卷积 -> 激活 -> 池化 
        x = self.conv1(x) #根据卷积的尺寸计算公式，计算结果是30，具体计算公式后面第二章第四节 卷积神经网络 有详细介绍。
        x = F.relu(x)
        print(x.size()) # 结果：[1, 6, 30, 30]
        x = F.max_pool2d(x, (2, 2)) #我们使用池化层，计算结果是15
        x = F.relu(x)
        print(x.size()) # 结果：[1, 6, 15, 15]
        # reshape，‘-1’表示自适应
        #这里做的就是压扁的操作 就是把后面的[1, 6, 15, 15]压扁，变为 [1, 1350]
        x = x.view(x.size()[0], -1) 
        print(x.size()) # 这里就是fc1层的的输入1350 
        x = self.fc1(x)        
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=1350, out_features=10, bias=True)
)


In [4]:
for parameters in net.parameters():
    print(parameters)

Parameter containing:
tensor([[[[-0.2428,  0.2286,  0.1983],
          [ 0.1831, -0.0351,  0.2653],
          [-0.2534,  0.0415,  0.2001]]],


        [[[-0.0763, -0.2245, -0.0219],
          [ 0.1429, -0.1048, -0.2159],
          [ 0.2123,  0.0350, -0.3094]]],


        [[[ 0.0324, -0.2759, -0.1003],
          [-0.1575,  0.1232, -0.1259],
          [-0.3282, -0.1823, -0.0512]]],


        [[[ 0.0680, -0.1849, -0.1992],
          [-0.3186, -0.0497,  0.2773],
          [-0.1001,  0.3318, -0.1587]]],


        [[[ 0.3165,  0.2350,  0.0535],
          [-0.3199, -0.1756, -0.1031],
          [ 0.1624,  0.3235, -0.3067]]],


        [[[ 0.1414,  0.2526,  0.1061],
          [-0.1497, -0.2813,  0.2771],
          [ 0.0252,  0.3055,  0.3318]]]], requires_grad=True)
Parameter containing:
tensor([ 0.0699, -0.0272, -0.2795,  0.2515,  0.1410,  0.2205],
       requires_grad=True)
Parameter containing:
tensor([[-0.0235,  0.0216, -0.0035,  ...,  0.0150, -0.0172, -0.0219],
        [ 0.0122, -0.0064,  0

In [5]:
for name, parameters in net.named_parameters():
    print(name, ':', parameters.size())

conv1.weight : torch.Size([6, 1, 3, 3])
conv1.bias : torch.Size([6])
fc1.weight : torch.Size([10, 1350])
fc1.bias : torch.Size([10])


In [6]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
out.size()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])


torch.Size([1, 10])

In [7]:
input.size()

torch.Size([1, 1, 32, 32])

In [8]:
net.zero_grad()
out.backward(torch.ones(1, 10))

In [9]:
y = torch.arange(0, 10).view(1, 10).float()
criterion = nn.MSELoss()
loss = criterion(out, y)

print(loss.item())

26.069942474365234


In [13]:
y

tensor([[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]])

In [15]:
out.size()

torch.Size([1, 10])

In [10]:
print(loss)

tensor(26.0699, grad_fn=<MseLossBackward>)


In [11]:
loss.item()

26.069942474365234

In [12]:
import torch.optim

In [16]:
out = net(input)
criterion = nn.MSELoss()
loss = criterion(out, y)

optimizer = torch.optim.SGD(net.parameters(), lr = 0.01)

optimizer.zero_grad()

loss.backward()

optimizer.step()

torch.Size([1, 1, 32, 32])
torch.Size([1, 6, 30, 30])
torch.Size([1, 6, 15, 15])
torch.Size([1, 1350])
